In [3]:
import csv
from datetime import datetime, timezone
import requests
import time
csv_list = [ 'django.django.csv', 'kubernetes.kubernetes.csv', 'elastic.elasticsearch.csv', 'rails.rails.csv', 'tensorflow.tensorflow.csv', 
            'electron.electron.csv', 'twbs.bootstrap.csv', "redis.redis.csv", 'microsoft.vscode.csv', 'facebook.react.csv']

##### feature index ##########
pr_author_login_idx = 0  ### delete
pr_author_name_idx = 1  ### replace with race
pr_create_at_idx = 2  ### replace with notice time
pr_merged_at_idx = 3  ### replace with review time
pr_changed_files_idx = 4
pr_LOC_idx = 5
pr_reviewers_idx = 6   #### delete for hypo 1-4 , change for hypo 5
pr_first_review_at_idx = 7  ### delete
pr_review_commits_idx = 8
pr_review_comments_idx = 9
pr_commits_idx = 10

repo_stars_idx = 11
repo_created_at_idx = 12  ####  replace with repo time
repo_external_contribution_idx = 13
repo_team_size_idx = 14

user_followers_idx = 15
user_created_at_idx = 16   ### replace with user time
user_experience_idx = 17
user_repo_experience_idx = 18
user_main_member_idx = 19
user_watched_idx = 20
user_succ_rate_idx = 21

def set_string(reveiewers):
    
    reveiewers = reveiewers[1:-1]
    reveiewer_list = reveiewers.split(", ")
    res = []
    for name in reveiewer_list:
        res.append(name[1:-1])
    return res

######### get the dictionary for user-name : race
race_dic = {} # key name, value race

# for file_name in csv_list:
#     f_read = open("./csv_files/" + file_name, 'r')
#     with f_read:
#         reader = csv.reader(f_read)
#         count = 0
#         for read_row in reader:
#             if count == 0:
#                 count += 1
#                 continue
#             if len(read_row) <= 1: continue
#             author_name = read_row[pr_author_name_idx]
#             race_dic[author_name] = "Unknown"
#             for reviewer_name in set_string(read_row[pr_reviewers_idx]):
#                 race_dic[reviewer_name] = "Unknown"
                


In [3]:
# black_list = ['Levvie%20-%20she/her', 'Ben%20Sheldon%20[he/him]']

# for name in race_dic:
#     full_name = name.replace(" ", "%20")
#     #print(full_name) White API
#     print(full_name)
#     if full_name in black_list:
#         continue
#     ethnic_req = requests.get("http://www.name-prism.com/api_token/eth/json/4868133984c85190/"+full_name).json()
#     for race, value in ethnic_req.items():
#         if value >= 0.8 and (race == "API" or race == "White" or race == "Hispanic"):
#             race_dic[name] = race
#             break
#     time.sleep(0.01)

# print(len(race_dic))

Mariusz%20Felisiak
Carlton%20Gibson
Hasan%20Ramezani
Nick%20Pope
David%20Smith
Chris%20Jerdonek
Bal%20Krishna%20Jha
Abhyudai
Rohith%20PR
Pēteris%20Caune
Keryn%20Knight
Jordan%20Bae
Jacob%20Walls
David%20Wobrock
Virtosu%20Bogdan
Paolo%20Melchiorre
Tim%20Graham
Tomasz%20Wójcik
Jonathan%20Davis
Asif%20Saif%20Uddin
Simon%20Charette
Wu%20Haotian
Claude%20Paroz
Demetris%20Stavrou
Daniele%20Procida
Unknown
Wilhelm%20Klopp
Wille%20Marcel
Ties%20Jan%20Hefting
Adam%20Johnson
Shai%20Berger
Florian%20Apolloner
Thomas%20Grainger
Eduardo%20Ahumada
Mateo%20Radman
Aymeric%20Augustin
Hannes%20Ljungberg
Thomas%20Güttler
Dan%20Strokirk
Russell%20Keith-Magee
Jon%20Dufresne
Shipeng%20Feng
Illia%20Volochii
Nicolás%20Restrepo
Jarek%20Głowacki
Ian%20Foote
Arya%20Khaligh
Takayuki%20Hirayama
Tom%20Hamilton%20Stubber
Igor%20Kuivjogi%20Fernandes
Mohammadreza%20Varasteh
Mike%20Lissner
Markus%20Holtermann
Girish%20Sontakke
Chiorufarewerin
Thibaud%20Colas
Tobias%20Bengfort
cflb
Rust%20Saiargaliev
Johannes%20Maron
Pa

David%20Stosik
Mark%20VanLandingham
Alexandre%20Ruban
Guillaume%20Briday
Ben%20Greenberg
Mike%20Rogers
Theo%20Julienne
John%20Bachir
Marek%20Kasztelnik
Muhammad%20Usman
Justin%20Robinson
Garrett%20Dimon
Nate%20Pinsky
Kyle%20Keesling
Alex%20Robbin
Ritikesh
Hu%20Hailin
Asherah%20Connor
aaron
Nate%20Matykiewicz
Ryota%20Egusa
Alan%20Savage
Paul%20Keen
Michael%20Pevzner
Pedro%20Baracho
Ross%20Kaffenberger
Karl%20Entwistle
Miguel%20Michelson%20Martinez
Markus
Matijs%20van%20Zuijlen
flprben
Bogdan%20Gusiev
Filipe%20Giusti
Louis%20Cloutier
Simon%20Perepelitsa
Clayton%20Smith
Jack%20McCracken
alpaca-tc
Dan%20Puttick
Jan%20Sandbrink
Shalvah
Rodrigo%20Andrés%20Contreras%20Vilina
Tahsin%20Hasan
David%20Angulo
Julien
Joel%20Hawksley
Alan%20Guo%20Xiang%20Tan
Andy%20Atkinson
Stephann%20Vasconcelos
Cauê%20Guerra
Jahfer%20Husain
Tomoki%20Sekiyama
Edu%20Depetris
Takumi%20Shotoku
Tomer%20Brisker
Juan%20Manuel%20Ramallo
Ngan%20Pham
Kir%20Shatrov
Alberto%20Almagro
Filipe%20Sabella
Sam%20Bostock
Michael%20O

Russell%20Power
Hollow%20Man
Gyeongjae%20Choi
Corey%20Cole
Andrzej%20Pomirski
Kacper%20Bąk
Aaron%20Mondal
Dmitry%20Volodin
Yuta%20Fukasawa
E
Jacob%20Valdez
Cloud%20Han
Xinan%20Jiang(姜曦楠)
Michael%20Kuchnik
Sergey%20Popov
Skye%20Wanderman-Milne
Wen-Heng%20(Jack)%20Chung
Stephan%20Herhut
Alexander%20Belyaev
Albert%20Villanova%20del%20Moral
Yuefeng%20Zhou
Alexander%20Bayandin
Geoffrey%20Martin-Noble
Benjamin%20Kramer
Jason%20Zaman
Pedro%20Gonnet
I%20Wayan%20Dharmana
Sungmann%20Cho
Wansoo%20Kim
firejq
Mateusz%20Hołenko
token
Frédéric%20Rechtenstein
Wenwen%20(Irvine)%20Ouyang
Soumyadip%20Sarkar
shuaidu
Kristian%20Hartikainen
yuanbopeng
Revan%20Sopher
Long%20M.%20Lưu%20(刘明龙)
Tyler%20Davis
Matthias
Yuqi%20Li
Paul%20Isaac's
Ian%20Beauregard
Tarandeep%20Singh
Thomas%20Joerg
Marat%20Kopytjuk
Maria%20Romanenko%20Vexlard
Marcos%20Pereira
Poedator
Brent%20M.%20Spell
Marcin%20Owsiany
sulenn
Yoav%20Ramon
Daun
Kasra%20Bigdeli
Pavithra%20Vijay
Ashwin%20Phadke
Rohan%20Lekhwani
Jakub%20Beránek
Mazhar
Paul

Tim%20Fenney
Chuck%20Lantz
Jason%20Rueckert
Krzysztof%20Cieślak
Nicholas%20Rayburn
Ashray%20Jha
Liangcheng%20Juves
Glaukio
Rachel%20Macfarlane
Hassan%20Sani
Daniel%20Gary
Geoff%20Paulsen
Eric%20Amodio
Vibhav
SNDST00M:%20M.U.N.I.N
Cody%20Beyer
Donald%20sullender
Ryan%20Zoeller
Ben%20McCann
Valeriy
Mohamed%20AbouGendy
Andrew%20Choi
Juliana%20Peña
Yongming%20Lai
Steven%20Degutis
Augusto%20Maneiro
Anton%20Kosyakov
Andrew%20Casey
Tim%20Maffett
Sana%20Ajani
Liao%20Jinyuan
Jade
Konstantin%20Solomatov
Yanis%20HAMITI
Anthony%20Martin
Pooya%20Vakeely
William%20Desgardin
Noah%20Bliss
Alex%20Myczko
Michael%20Plainer
Wendell
Daniel%20Knights
Abhijeet%20Joshi
Danylo%20Shevchenko
Stefan%20Petrescu
Brandon%20Waterloo%20[MSFT]
Simon%20Sobisch
Hsuan-An%20Weng%20Lin
Nate%20Moore
ABHIJEET%20PANDEY
Hanxiao%20Liu
Tanner%20Bennett
T.Y.M.SAI
Stanislav
Andrew%20Scott
Felix%20Haase
Shunsuke%20Iwamoto
Paul%20Adams
Manuel%20Thalmann
Ted%20Spence
Anton%20Volokha
Utku%20Gultopu
Habib%20Karim
Alexandre%20Schnegg
Kev

In [4]:
# import json

# with open('name_race_dic.json', 'w') as outfile:
#         json.dump(race_dic, outfile)

# api_count = 0
# white_count = 0
# hispanic_count = 0
# for name in race_dic:
#     if race_dic[name] == "API":
#         api_count += 1
#     elif race_dic[name] == "White":
#         white_count += 1
#     elif race_dic[name] == "Hispanic":
#         hispanic_count += 1

# print("here are {} API users among {} users".format(api_count, len(race_dic)))
# print("there are {} White users among {} users".format(white_count, len(race_dic)))
# print("there are {} Hispanic users among {} users".format(hispanic_count, len(race_dic)))


here are 398 API users among 2020 users
there are 1277 White users among 2020 users
there are 74 Hispanic users among 2020 users


In [4]:
import json

f = open('name_race_dic.json')
race_dic = json.load(f)

api_count = 0
white_count = 0
hispanic_count = 0
for name in race_dic:
    if race_dic[name] == "API":
        api_count += 1
    elif race_dic[name] == "White":
        white_count += 1
    elif race_dic[name] == "Hispanic":
        hispanic_count += 1

print("There are {} API users among {} users".format(api_count, len(race_dic)))
print("There are {} White users among {} users".format(white_count, len(race_dic)))
print("There are {} Hispanic users among {} users".format(hispanic_count, len(race_dic)))

here are 398 API users among 2020 users
there are 1277 White users among 2020 users
there are 74 Hispanic users among 2020 users


In [24]:
##### build the data for hypo 1-4 ##########

### the new variables for model
pr_head1 = ['pr_author_race','pr_notice_time','pr_review_time','pr_changed_files',
    'pr_LOC','pr_review_commits','pr_review_comments','pr_commits'] # 8


repo_head = ['repo_stars','repo_create_time','repo_external_contribution','repo_team_size'] # 4

user_head = ['user_followers','user_dev_time','user_experience', 'user_repo_experience','user_main_member',
             'user_watched','user_succ_rate'] # 7

csv_head1 = pr_head1 + repo_head + user_head

length = len(csv_head1)
cur_time = datetime.now(timezone.utc)


with open("combine_data.csv", 'w') as f_write:
    csv_write = csv.writer(f_write)
    csv_head = csv_head1
    csv_write.writerow(csv_head)
    
    for file_name in csv_list:
        f_read = open("./csv_files/" + file_name, 'r')
        with f_read:
            reader = csv.reader(f_read)
            count = 0
            for read_row in reader:
                if count == 0:
                    count += 1
                    continue
                if len(read_row) <= 1: continue
                write_row = []
                
                
                ######## calculate the time difference ##########
                created_at = datetime.strptime(read_row[pr_create_at_idx], "%Y-%m-%dT%H:%M:%S%z")
                merged_at = datetime.strptime(read_row[pr_merged_at_idx], "%Y-%m-%dT%H:%M:%S%z") 
                review_at =  datetime.strptime(read_row[pr_first_review_at_idx] + "Z", "%Y-%m-%d %H:%M:%S%z") 
                repo_created_at = datetime.strptime(read_row[repo_created_at_idx], "%Y-%m-%dT%H:%M:%S%z") 
                user_created_at = datetime.strptime(read_row[user_created_at_idx], "%Y-%m-%dT%H:%M:%S%z") 
                
                pr_notice_time = review_at - created_at
                pr_review_time = merged_at - review_at
                repo_create_time = cur_time - repo_created_at
                user_create_time = cur_time - user_created_at
                ######### get the author race ##########
                author_name = read_row[pr_author_name_idx]
                pr_author_race = race_dic[author_name]
                if pr_author_race == "Unknown" or pr_author_race == "Hispanic" : continue
                ## pr features
                write_row.append(pr_author_race)
                write_row.append(min(pr_notice_time.days, 14))
                write_row.append(min(pr_review_time.days, 14))
                write_row.append(
                    min(int(read_row[pr_changed_files_idx]), 30)
                )
                write_row.append(
                    min(int(read_row[pr_LOC_idx]), 1500)
                )
                write_row.append(read_row[pr_review_commits_idx])
                write_row.append(read_row[pr_review_comments_idx])
                write_row.append(read_row[pr_commits_idx])
                
                ## repo features
                star_k = int(read_row[repo_stars_idx]) / 1000.0
                create_year = repo_create_time.days / 365.0
                write_row.append(star_k)
                write_row.append(create_year)
                write_row.append(read_row[repo_external_contribution_idx])
                write_row.append(read_row[repo_team_size_idx])

                ## user features
                followers_num = int(read_row[user_followers_idx])
                followers_num = min(followers_num, 500)
                write_row.append(followers_num)
                write_row.append(user_create_time.days / 365.0)
                write_row.append(read_row[user_experience_idx])
                write_row.append(read_row[user_repo_experience_idx])
                write_row.append(read_row[user_main_member_idx])
                write_row.append(read_row[user_watched_idx])
                write_row.append(read_row[user_succ_rate_idx])
                
                csv_write.writerow(write_row)
                

In [23]:
##### build the data for hypo 5 ##########
pr_head5 = ['pr_author_reviewer_race','pr_notice_time','pr_review_time','pr_changed_files',
    'pr_LOC','pr_review_commits','pr_review_comments','pr_commits'] # 9

#pr_reviewer_race

csv_head5 = pr_head5 + repo_head + user_head


api_api_count = 0
total_count = 0
with open("combine_data_h5.csv", 'w') as f_write:
    csv_write = csv.writer(f_write)
    csv_head = csv_head5
    csv_write.writerow(csv_head)
    
    for file_name in csv_list:
        f_read = open("./csv_files/" + file_name, 'r')
        with f_read:
            reader = csv.reader(f_read)
            count = 0
            for read_row in reader:
                if count == 0:
                    count += 1
                    continue
                if len(read_row) <= 1: continue
                write_row = []

                
                ######## calculate the time difference ##########
                created_at = datetime.strptime(read_row[pr_create_at_idx], "%Y-%m-%dT%H:%M:%S%z")
                merged_at = datetime.strptime(read_row[pr_merged_at_idx], "%Y-%m-%dT%H:%M:%S%z") 
                review_at =  datetime.strptime(read_row[pr_first_review_at_idx] + "Z", "%Y-%m-%d %H:%M:%S%z") 
                repo_created_at = datetime.strptime(read_row[repo_created_at_idx], "%Y-%m-%dT%H:%M:%S%z") 
                user_created_at = datetime.strptime(read_row[user_created_at_idx], "%Y-%m-%dT%H:%M:%S%z") 
                
                pr_notice_time = review_at - created_at
                pr_review_time = merged_at - review_at
                repo_create_time = cur_time - repo_created_at
                user_create_time = cur_time - user_created_at
                ######### get the author and reviewer race ##########
                author_name = read_row[pr_author_name_idx]
                pr_author_race = race_dic[author_name]
                if pr_author_race == "Unknown" or pr_author_race == "Hispanic" : continue
                
                ######### unique
                reviewer_list = set_string(read_row[pr_reviewers_idx])
                if len(reviewer_list) != 1:
                    continue
                reviewer_race = race_dic[reviewer_list[0]]
                if reviewer_race == "Unknown" or reviewer_race == "Hispanic" : continue
                
                total_count += 1
                if pr_author_race == "API" and reviewer_race == "API":
                    api_api_count += 1
                author_reviewer_race_pair = pr_author_race + "-" + reviewer_race
                ######### unique
                
                ## pr features 
                write_row.append(author_reviewer_race_pair)
                write_row.append(min(pr_notice_time.days, 14))
                write_row.append(min(pr_review_time.days, 14))
                write_row.append(
                    min(int(read_row[pr_changed_files_idx]), 30)
                )
                write_row.append(
                    min(int(read_row[pr_LOC_idx]), 1500)
                )
                write_row.append(read_row[pr_review_commits_idx])
                write_row.append(read_row[pr_review_comments_idx])
                write_row.append(read_row[pr_commits_idx])
                
                
                ## repo features
                star_k = int(read_row[repo_stars_idx]) / 1000.0
                create_year = repo_create_time.days / 365.0
                write_row.append(star_k)
                write_row.append(create_year)
                write_row.append(read_row[repo_external_contribution_idx])
                write_row.append(read_row[repo_team_size_idx])

                ## user features
                followers_num = int(read_row[user_followers_idx])
                followers_num = min(followers_num, 500)
                write_row.append(followers_num)
                write_row.append(user_create_time.days / 365.0)
                write_row.append(read_row[user_experience_idx])
                write_row.append(read_row[user_repo_experience_idx])
                write_row.append(read_row[user_main_member_idx])
                write_row.append(read_row[user_watched_idx])
                write_row.append(read_row[user_succ_rate_idx])
                
                csv_write.writerow(write_row)
                
print(api_api_count)
print(total_count)

73
4442
